# Práctica de Sistemas Recomendadores: pyreclab - SVD e Implicit Feedback

En este práctico, volveremos a utilizar la biblioteca de Python [pyreclab](https://github.com/gasevi/pyreclab), desarrollado por los Laboratorios IALab y SocVis de la Pontificia Universidad Católica de Chile, para aprender sobre algoritmos más avanzados de recomendación:

* Singular Value Decomposition (SVD)
* Alternating Least Squares (ALS) Implicit Feedback  Coordinate Descent

**Basado en el material original de**: Denis Parra, Gabriel Sepúlveda

**Ayudantes**: Manuel Cartagena, Andrés Carvallo y Patricio Cerda. 




Nombre: **completar**

# Índice

>[Práctica de Sistemas Recomendadores 2: pyreclab - SVD e Implicit Feedback](#scrollTo=NC-ceGb8LRLT)

>[Índice](#scrollTo=l-3HVp9guEsg)

>[Actividad 1](#scrollTo=uKAqmo5IdQFI)

>[Descargando la información](#scrollTo=IFpEoacrMwQx)

>[Revisar archivos descargados](#scrollTo=TJon9T5ZMwRG)

>>[Como ver la información de una o más películas](#scrollTo=WkU90OXOh-pk)

>>[Graficar la información](#scrollTo=_tjnQ5koSAup)

>>[Preparar entorno](#scrollTo=7HU7NoDUhnYl)

>[SVD](#scrollTo=cKYbD4T-OCtj)

>>[Entrenar SVD](#scrollTo=JxvKwB2Su1GZ)

>>[Testear predicción](#scrollTo=lIv2YBYKva85)

>>[Testear recomendaciones](#scrollTo=UYEX5bWsvnYJ)

>>[Aplicar SVD al usuario en cuestión](#scrollTo=_RNJRHp2vwIQ)

>>>[Graficar los géneros de las películas para ver si se asemejan a lo visto anteriormente](#scrollTo=LAs9AxrAv1MR)

>>>[Actividad 2](#scrollTo=ny4E5JcYZiyW)

>>>[Actividad 3](#scrollTo=BZvaluMLg88m)

>[ALS (Implicit Feedback)  Coordinate Descent](#scrollTo=lWtg8NYNXFJ_)

>>[Crear objeto ALS](#scrollTo=9I40w0XT0ccx)

>>[Entrenar ALS](#scrollTo=f0h71UFdzOjw)

>>[Calcular métricas de recomendación (MAP y NDCG)](#scrollTo=OzAsvtPg0XXi)

>>[Recomendar a un usuario en particular](#scrollTo=M7PizyiSuAGS)

>>[Actividad 4](#scrollTo=aJdHEECCxGUb)



# Actividad 1

Antes de empezar con la actividad, responder la siguiente pregunta con lo visto en clases

**Pregunta:** Explique cómo funciona SVD (como modelo teórico, no piense en la implementación), y cómo funciona el método `train()` en  `pyreclab`. En particular explique

- ¿Qué datos recibe SVD? (qué tipo de columnas)
- ¿Que hace con esos datos? Si realiza algún calculo, ¿Qué calculo?
- ¿Cómo recomienda este modelo?
- En `pyreclab`, ¿qué hace el método `train()`? ¿Calcula información?, ¿no hace nada?, ¿ordena datos? Explique como debería funcionar dado el modelo teórico.






**Respuesta:** COMPLETAR

# Descargando la información

Vaya ejecutando cada celda presionando el botón de **Play** o presionando Ctrl+Enter (Linux y Windows) o Command+Enter (Macosx) para descargar las bases de datos

In [0]:
!curl -L -o "u2.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

In [0]:
!curl -L -o "u2.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

In [0]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

Los archivos **u1.base** y **u1.test** tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [movielens](https://grouplens.org/datasets/movielens/).

# Revisar archivos descargados

Revisemos cómo es uno de estos archivos:


In [0]:
# Primero creamos el dataframe con los datos
df_train = pd.read_csv('u2.base',
                         sep='\t',
                         names=['userid', 'itemid', 'rating', 'timestamp'],
                         header=None)
df_train.head()

In [0]:
# Ahora queremos realizar una observación rápida de los datos

df_train.describe()

Por otra parte, para obtener información adicional de cada película tal como **título**, **fecha de lanzamiento**, **género**, etc., cargaremos el archivo de items descargado (`u.item`) para poder mapear cada identificador de ítem al conjunto de datos que lo describe. Revisemos el contenido de este archivo

In [0]:
columns = ['movieid', 'title', 'release_date', 'video_release_date', \
           'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', \
           'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', \
           'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', \
           'Thriller', 'War', 'Western']

In [0]:
# Cargamos el dataset con los items
df_items = pd.read_csv('u.item',
                        sep='|',
                        index_col=0,
                        names = columns,
                        header=None, 
                        encoding='latin-1')
df_items.head()

In [0]:
# Realizamos una inspección sobre estos datos (.info())

df_items.info()


## Como ver la información de una o más películas
Para esto seguir los siguientes pasos, 
1. Determinar los índices de las películas
2. Pedir a `info_file` las columnas
3. (adicional) Indicar alguna columna en específico

In [0]:
# ejemplo de cómo visualizar titulos de peliculas en base a sus IDs
pelis = [5,4,1]
info_file.loc[pelis]

Para éste código:
1. `pelis = [5,4,1]` indica que queremos las películas cuyo índice son 5, 4 y 1
2. `info_file.loc[pelis]` el método `loc` permite acceder a esas columna
A continuación se verá como obtener una columna en específico. Para esto solo es necesario agregar corchetes y el nombre de la columna

In [0]:
info_file.loc[pelis]['title']

## Graficar la información

Generemos algunos gráficos en relación a las películas.

In [0]:
import altair as alt
import pandas as pd

genre_columns = ['unknown', 'Action', 'Adventure', 'Animation', \
           'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', \
           'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', \
           'Thriller', 'War', 'Western']


genre_count = pd.DataFrame(df_items[genre_columns].sum().sort_values())

genre_count = genre_count.reset_index()
genre_count.columns = ["genre", "value"]


alt.Chart(genre_count).mark_bar().encode(
    x=alt.X('value'),
    y=alt.Y("genre", sort=alt.EncodingSortField("value", order='ascending'))
).interactive()

## Preparar entorno
Primero es necesario instalar una librería para realizar recomendaciones. Esta se llama ***pyreclab***

In [0]:
!pip install pyreclab --upgrade

Luego necesitamos importar las librerías a utilizar en este práctico

In [0]:
import pandas as pd
import pyreclab
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt

%matplotlib inline

# SVD

Tome un usuario, cualquiera, del dataset y vamos a reportar parte de sus **gustos explícitamente** (es decir, nombre de las películas, género, etc., no id). Para este mismo usuario, la actividad será que **revise explícitamente** las recomendaciones al llegar a la parametrización que reporta las **mejores métricas**.

En este caso pusimos el número **2** pero puede poner otro. Primero vamos a mostrar cada película que ha visto este usuario.

In [0]:
user_id = 2
user_df = df_train[df_train['userid'] == user_id]



Vaoms a visualizar los géneros de las películas que ha visto el usuario.

In [0]:
import altair as alt
import pandas as pd

genre_columns = ['unknown', 'Action', 'Adventure', 'Animation', \
           'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', \
           'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', \
           'Thriller', 'War', 'Western']

df_movies_user = df_items.loc[user_df['itemid'].values]

genre_count = pd.DataFrame(df_movies_user[genre_columns].sum().sort_values())

genre_count = genre_count.reset_index()
genre_count.columns = ["genre", "value"]


alt.Chart(genre_count).mark_bar().encode(
    x=alt.X('value'),
    y=alt.Y("genre", sort=alt.EncodingSortField("value", order='ascending'))
).interactive()

## Entrenar SVD

Ahora vamos a definir el objeto **svd** para entrenarlo y luego recomendar. Los parámetros que escogimos fue:

- `factores = 100`

- `maxiter = 100`

- `lr = 0.01`

- `lamb = 0.1`

In [0]:
# Definicion de objeto svd

svd = pyreclab.SVD(dataset='u2.base',
                   dlmchar=b'\t',
                   header=False,
                   usercol=0,
                   itemcol=1,
                   ratingcol=2)

# Entrenamiento del modelo
svd.train(factors=100, maxiter=100, lr=0.01, lamb=0.1)

## Testear predicción

Ahora vamos a ver el MAE y RMSE con los parámetros anteriormente definidos para el entrenamiento para predecir _rating_.

In [0]:
# Testing de predicciones
predlist, mae, rmse = svd.test(input_file='u2.test',
                               dlmchar=b'\t',
                               header=False,
                               usercol=0,
                               itemcol=1,
                               ratingcol=2)

print('MAE: {}\nRMSE: {}'.format(mae, rmse))

## Testear recomendaciones

Ahora vamos a ver el MAE y RMSE con los parámetros anteriormente definidos para el entrenamiento para recomendar nuevas películas.

In [0]:
# Testing de recomendaciones
top_n = 20

recommendList, maprec, ndcg = svd.testrec(input_file='u2.test',
                                          dlmchar=b'\t',
                                          header=False,
                                          usercol=0,
                                          itemcol=1,
                                          ratingcol=2,
                                          topn=top_n,
                                          relevance_threshold=2,
                                          includeRated=False)

print('MAP: {}\nNDCG@{}: {}'.format(maprec, top_n, ndcg))

## Aplicar SVD al usuario en cuestión

In [0]:
# Calcular las recomendaciones para el usuario escogido
ranking = [int(r) for r in svd.recommend(str(user_id), top_n, includeRated=False)]
print('Recommendation for user {}: {}'.format(user_id, ranking))

In [0]:
# Ver explicitamente las recomendaciones para un usuario
df_items.loc[ranking]

### Graficar los géneros de las películas para ver si se asemejan a lo visto anteriormente

In [0]:
import altair as alt
import pandas as pd

genre_columns = ['unknown', 'Action', 'Adventure', 'Animation', \
           'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', \
           'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', \
           'Thriller', 'War', 'Western']

df_movies_user = df_items.loc[ranking]

genre_count = pd.DataFrame(df_movies_user[genre_columns].sum().sort_values())

genre_count = genre_count.reset_index()
genre_count.columns = ["genre", "value"]


alt.Chart(genre_count).mark_bar().encode(
    x=alt.X('value'),
    y=alt.Y("genre", sort=alt.EncodingSortField("value", order='ascending'))
).interactive()

### Actividad 2

Pruebe distintos valores de los parámetros de entrenamiento del algoritmo SVD:

- Número de factores
- Número de iteraciones máxima
- Learning rate
- Lambda

Finalmente describa cómo estos afectan las métricas y las recomendaciones hechas para el usuario que escogió. En particular 

(1) Genere un nuevo objeto SVD

(2) Entrene con otros parámetros 

(3) Calcule RMSE y MAE para predicción

(4) Calcule RMSE y MAE para recomendar

(5) Recomienda 10 películas y **muestrelas explicitamente** 

(6) Grafique los géneros de esas 10 películas.

(7) Explique, con sus palabras, como cambiar estas métricuas afecta las recomendaciones.


In [0]:
# codigo para hacer los puntos 1, 2 y 3

In [0]:
# Código para hacer el punto 4

In [0]:
# Código para hacer el punto 5

In [0]:
# Código para hacer el punto 6

**respuesta del punto 7**

### Actividad 3

**Visualizar rendimiento de parámetros**

Genere un gráfico explorando todos los parámetros que ofrece el modelo `SVD`, donde se grafiquen los valores probados para cada parámetro, el RMSE y MAE obtenido para cada uno. Debe probar una cantidad razonable de valores buscando la mejor parametrización y entregar el notebook con dicha parametrización.

A continuación se muestra como se obtiene el RMSE y MAE cuando se cambia el parámetro de número de factores. En esta actividad se debe hacer lo mismo, pero cambian los otros 3 parámetros.



In [0]:
# Código para probar con diferentes Número de factores

factor_values = [100, 150, 200] # definir distintos numeros de factores
mae_values = [] # para almacenar resultados de MAP
rmse_values = [] # para almacenar valores de NDCG

for f in factor_values:
  svd.train(factors=f, maxiter=100, lr=0.01, lamb=0.1)
  
  predlist, mae, rmse = svd.test(input_file='u2.test',
                                 dlmchar=b'\t',
                                 header=False,
                                 usercol=0,
                                 itemcol=1,
                                 ratingcol=2)
  
  mae_values.append(mae)
  rmse_values.append(rmse)

print(mae_values)
print(rmse_values)

In [0]:
# Código para visualizar MAE VS Número de factores
plt.plot(factor_values, mae_values, 'r-')
plt.show()

In [0]:
# Código para visualizar RMSS VS Número de factores

plt.plot(factor_values, rmse_values, 'bo-')
plt.show()

In [0]:
# Código para probar con diferentes Número de iteraciones máxima 


In [0]:
# Código para visualizar RMSE VS Número de iteraciones máxima 


In [0]:
# Código para visualizar MAE VS Número de iteraciones máxima 


In [0]:
# Código para probar con diferentes Learning Rate


In [0]:
# Código para visualizar RMSE VS Learning Rate


In [0]:
# Código para visualizar MAE VS Learning Rate


In [0]:
# Código para probar con diferentes Lambda

In [0]:
# Código para visualizar RMSE VS Lambda


In [0]:
# Código para visualizar MAE VS Lambda


# ALS (Implicit Feedback)  Coordinate Descent     _(Opcional)_



## Crear objeto ALS

In [0]:
%%time
# Definicion del objeto recomendador als
alscg = pyreclab.IFAlsConjugateGradient(dataset='u2.base',
                     dlmchar=b'\t',
                     header=False,
                     usercol=0,
                     itemcol=1,
                     observationcol=2)

## Entrenar ALS

**Importante** La siguiente casilla toma al menos **1 minuto** en ejecutar. Por lo tanto debe esperar un poco.

In [0]:
%%time
# Entrenamiento del modelo
alscg.train(factors=120, alsNumIter=10, lambd=0.001, cgNumIter=3)

## Calcular métricas de recomendación (MAP y NDCG)

In [0]:
%%time
# Testing de recomendaciones en el dataset de testx
top_n = 20

recommendList, maprec, ndcg = alscg.testrec(input_file='u2.test',
                                          dlmchar=b'\t',
                                          header=False,
                                          usercol=0,
                                          itemcol=1,
                                          ratingcol=2,
                                          topn=top_n,
                                          relevance_threshold=2,
                                          includeRated=False)

print('MAP: {}\nNDCG@{}: {}'.format(maprec, top_n, ndcg))

## Recomendar a un usuario en particular

Listar recomendaciones para un usuario en particular. En este caso se escogió el número **10**.

In [0]:
%%time
# Calcular las recomendaciones para el usuario escogido
user_id = 10
ranking = [int(r) for r in alscg.recommend(str(user_id), top_n, includeRated=False)]
print('Recommendation for user {}: {}'.format(user_id, ranking))

In [0]:
# Ver explicitamente las recomendaciones para el usuario
df_items.loc[ranking]